In [2]:
import numpy as np
from scipy import misc
import glob
import os, os.path
from tensorflow.contrib.data import Dataset, Iterator
import re

In [3]:
file_dir = './'
frame_dir = file_dir + 'frames/'
label_dir = file_dir + 'labels/'
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [4]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import time
from scipy.misc import imread
from scipy.misc import imresize
import matplotlib.image as mpimg
from scipy.ndimage import filters
import urllib
from numpy import random

import tensorflow as tf

In [5]:
data_env= '.'
frames_path=os.path.join(data_env,'frames');
label_path=os.path.join(data_env,'labels');

def load_data(start_path):
    X = []
    FA = []
    ges = []
    obj = []
    head = []
    print(start_path)
    for dir_name in os.listdir(start_path): # 'office', 'lab', 'house'
        loc_path = os.path.join(start_path, dir_name)
        for num in os.listdir(loc_path): # '1', '2', '3'
            loc_num_path = os.path.join(loc_path, num)
            for pos in os.listdir(loc_num_path): # 'Lhand', 'Rhand', 'head'
                loc_num_pos_path = os.path.join(loc_num_path, pos)
                file_names = os.listdir(loc_num_pos_path)
                file_names = sorted(file_names, key=lambda x: int(re.sub('\D', '', x)))
                file_names = [os.path.join(loc_num_pos_path,f) for f in file_names]
                if pos[0] == 'L':
                    X += file_names
                    if 'train' in start_path:
                        FA += np.load(os.path.join(label_path, dir_name, 'FA_left'+num+'.npy')).astype(int).tolist()
                        ges += np.load(os.path.join(label_path, dir_name, 'ges_left'+num+'.npy')).astype(int).tolist()
                        obj += np.load(os.path.join(label_path, dir_name, 'obj_left'+num+'.npy')).astype(int).tolist()
                    elif 'test' in start_path:
                        if 'lab' in loc_num_pos_path:
                            offset = 4
                        else:
                            offset = 3
                        FA += np.load(os.path.join(label_path, dir_name, 'FA_left'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                        ges += np.load(os.path.join(label_path, dir_name, 'ges_left'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                        obj += np.load(os.path.join(label_path, dir_name, 'obj_left'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                elif pos[0] == 'R':
                    X += file_names
                    if 'train' in start_path:
                        FA += np.load(os.path.join(label_path, dir_name, 'FA_right'+num+'.npy')).astype(int).tolist()
                        ges += np.load(os.path.join(label_path, dir_name, 'ges_right'+num+'.npy')).astype(int).tolist()
                        obj += np.load(os.path.join(label_path, dir_name, 'obj_right'+num+'.npy')).astype(int).tolist()
                    elif 'test' in start_path:
                        if 'lab' in loc_num_pos_path:
                            offset = 4
                        else:
                            offset = 3
                        FA += np.load(os.path.join(label_path, dir_name, 'FA_right'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                        ges += np.load(os.path.join(label_path, dir_name, 'ges_right'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                        obj += np.load(os.path.join(label_path, dir_name, 'obj_right'+str(int(num)+offset)+'.npy')).astype(int).tolist()
                else:
                    head += file_names
                    head += file_names
    
    assert len(X) == len(FA) == len(ges) == len(obj) == len(head)
    return np.array([X, FA, ges, obj, head])


In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#xt_head, xt_lhand, xt_rhand, yt = load_data()

In [117]:
class Alex_cls:
    def __init__(self, hps):
        self.hps = hps
        
        self._build_inputs()
        self._build_model()
        self._get_saver()
        
    def _build_inputs(self):
        self.img = tf.placeholder(tf.float32, [None, self.hps.h_input, self.hps.w_input, 3])
        self.FA = tf.placeholder(dtype=tf.int32, shape=[None])
        self.ges = tf.placeholder(dtype=tf.int32, shape=[None])
        self.obj = tf.placeholder(dtype=tf.int32, shape=[None])
        
        # convert the label to one-hot encoding
        self.one_hot_FA = tf.one_hot(self.FA, 2)
        self.one_hot_ges = tf.one_hot(self.ges, 13)
        self.one_hot_obj = tf.one_hot(self.obj, 24)
    
    def _build_model(self):
        with tf.variable_scope('conv1') as scope:
            kernel = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[3, 3, 3, 16]))
            conv = tf.nn.conv2d(self.img, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases', [kernel.shape[-1]], initializer=tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv = tf.nn.relu(pre_activation, name=scope.name)

            # pool1
            pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool')
            # norm1
            norm = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm')

        # conv2
        with tf.variable_scope('conv2') as scope:
            kernel = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[3, 3, 16, 32]))
            conv = tf.nn.conv2d(norm, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases', [kernel.shape[-1]], initializer=tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv = tf.nn.relu(pre_activation, name=scope.name)

            # pool2
            pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool')
            # norm2
            norm = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm')

        # conv3
        with tf.variable_scope('conv3') as scope:
            kernel = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[3, 3, 32, 32]))
            conv = tf.nn.conv2d(norm, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases', [kernel.shape[-1]], initializer=tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv = tf.nn.relu(pre_activation, name=scope.name)

            # pool3
            pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool')
            # norm3
            norm = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm')
        
        # conv4
        with tf.variable_scope('conv4') as scope:
            kernel = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[3, 3, 32, 32]))
            conv = tf.nn.conv2d(norm, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.get_variable('biases', [kernel.shape[-1]], initializer=tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv = tf.nn.relu(pre_activation, name=scope.name)

            # pool4
            pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool')
            # norm4
            norm = tf.nn.lrn(pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm')
            
        # local3 shared fcn layer
        with tf.variable_scope('local3') as scope:
            # Move everything into depth so we can perform a single matrix multiply.
            reshape = tf.reshape(norm, [-1, 1024])
            weights = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[1024, 192]))
            biases = tf.get_variable('biases', [192], initializer=tf.constant_initializer(0.1))
            local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)

        with tf.variable_scope('softmax_linear_FA') as scope:
            weights = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[192, 2]))
            biases = tf.get_variable('biases', [2], initializer=tf.constant_initializer(0.1))
            softmax_linear_FA = tf.add(tf.matmul(local3, weights), biases, name=scope.name)
            self.output_FA = tf.nn.softmax(softmax_linear_FA)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
                    labels=self.one_hot_FA, logits=softmax_linear_FA, name='cross_entropy')
            cross_entropy_mean_FA = tf.reduce_mean(cross_entropy, name='cross_entropy_mean')

        with tf.variable_scope('softmax_linear_ges') as scope:
            weights = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[192, 13]))
            biases = tf.get_variable('biases', [13], initializer=tf.constant_initializer(0.1))
            softmax_linear_ges = tf.add(tf.matmul(local3, weights), biases, name=scope.name)
            self.output_ges = tf.nn.softmax(softmax_linear_ges)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
                    labels=self.one_hot_ges, logits=softmax_linear_ges, name='cross_entropy')
            cross_entropy_mean_ges = tf.reduce_mean(cross_entropy, name='cross_entropy_mean')

        with tf.variable_scope('softmax_linear_obj') as scope:
            weights = tf.get_variable('weights', initializer=tf.truncated_normal(shape=[192, 24]))
            biases = tf.get_variable('biases', [24], initializer=tf.constant_initializer(0.1))
            softmax_linear_obj = tf.add(tf.matmul(local3, weights), biases, name=scope.name)
            self.output_obj = tf.nn.softmax(softmax_linear_obj)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
                    labels=self.one_hot_obj, logits=softmax_linear_obj, name='cross_entropy')
            cross_entropy_mean_obj = tf.reduce_mean(cross_entropy, name='cross_entropy_mean')

        #calculate loss and training operation
        self.loss = cross_entropy_mean_FA*2 + cross_entropy_mean_ges*13 + cross_entropy_mean_obj*24
        
        self.global_step = tf.Variable(0, name="global_step", trainable=False)
        
        optimizer = tf.train.AdamOptimizer(self.hps.lr)
        grads_and_vars = optimizer.compute_gradients(self.loss)
        self.train_op = optimizer.apply_gradients(grads_and_vars, global_step=self.global_step)
    
    def build_data(self):
        #width = 32
        #height = 18
        width = 128
        height = 64
        
        # load dataset
        X, FA, ges, obj, X_head = load_data(os.path.join('./frames/train'))
        np.random.seed(0)
        idx = np.random.choice(len(X), size=int(len(X)*0.3), replace=False).astype(int)
        valid_idx = np.zeros(len(X), dtype=bool)
        valid_idx[idx] = True
        train_idx = np.ones(len(X), dtype=bool)
        train_idx[idx] = False
       
        print('num X_train:',len(X))

        X_train_path = tf.constant(X[train_idx])
        X_train_FA = tf.constant(FA[train_idx])
        X_train_ges = tf.constant(ges[train_idx])
        X_train_obj = tf.constant(obj[train_idx])
        X_train_head_path = tf.constant(X_head[train_idx])
        X_valid_path = tf.constant(X[valid_idx])
        X_valid_FA = tf.constant(FA[valid_idx])
        X_valid_ges = tf.constant(ges[valid_idx])
        X_valid_obj = tf.constant(obj[valid_idx])
        X_valid_head_path = tf.constant(X_head[valid_idx])
        self.num_train = X_train_path.shape[0].value
        self.num_valid = X_valid_path.shape[0].value


        # create TensorFlow Dataset objects
        dataset = Dataset.from_tensor_slices((X_train_path, X_train_FA, X_train_ges, X_train_obj, X_train_head_path))
        valid_dataset = Dataset.from_tensor_slices((X_valid_path, X_valid_FA, X_valid_ges, X_valid_obj, X_valid_head_path))
        
        
        def data_generator(X_train_path, X_train_FA, X_train_ges, X_train_obj, X_train_head_path):
            # read the img from file
            img_file = tf.read_file(X_train_path)
            img = tf.image.decode_image(img_file, channels=3)
            img = tf.image.convert_image_dtype(img, tf.float64)
            img.set_shape([1080, 1920, 3])
            img = tf.image.resize_images(img, size=[self.hps.h_input,self.hps.w_input])

            # read the img from file
            img_file = tf.read_file(X_train_head_path)
            img_head = tf.image.decode_image(img_file, channels=3)
            img_head = tf.image.convert_image_dtype(img_head, tf.float64)
            img_head.set_shape([1080, 1920, 3])
            img_head = tf.image.resize_images(img_head, size=[self.hps.h_input,self.hps.w_input])


            return img, X_train_FA, X_train_ges, X_train_obj, img_head

        dataset = dataset.map(data_generator,num_threads=4, output_buffer_size=10*self.hps.batch_size)
        dataset = dataset.batch(self.hps.batch_size)
        
        valid_dataset = valid_dataset.map(data_generator,num_threads=4, output_buffer_size=3*self.hps.batch_size)
        valid_dataset = valid_dataset.batch(self.hps.batch_size)

        # # create TensorFlow Iterator object
        iterator = Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
        self.next_element = iterator.get_next()

        valid_iterator = Iterator.from_structure(valid_dataset.output_types,
                                           valid_dataset.output_shapes)
        self.valid_next_element = valid_iterator.get_next()
        

        # # create two initialization ops to switch between the datasets
        self.training_init_op = iterator.make_initializer(dataset)
        self.validating_init_op = valid_iterator.make_initializer(valid_dataset)
    def build_test_data(self):
        width = 128
        height = 64
        X_test_path, X_test_FA, X_test_ges, X_test_obj, X_test_head_path = load_data(os.path.join('./frames/test'))
        print('num X_test:',len(X_test_path))
        
        self.num_test = len(X_test_path)
        test_dataset = Dataset.from_tensor_slices((X_test_path, X_test_FA, X_test_ges, X_test_obj, X_test_head_path))
        
        def data_generator(X_train_path, X_train_FA, X_train_ges, X_train_obj, X_train_head_path):
            # read the img from file
            img_file = tf.read_file(X_train_path)
            img = tf.image.decode_image(img_file, channels=3)
            img = tf.image.convert_image_dtype(img, tf.float64)
            img.set_shape([1080, 1920, 3])
            img = tf.image.resize_images(img, size=[self.hps.h_input,self.hps.w_input])

            # read the img from file
            img_file = tf.read_file(X_train_head_path)
            img_head = tf.image.decode_image(img_file, channels=3)
            img_head = tf.image.convert_image_dtype(img_head, tf.float64)
            img_head.set_shape([1080, 1920, 3])
            img_head = tf.image.resize_images(img_head, size=[self.hps.h_input,self.hps.w_input])


            return img, X_train_FA, X_train_ges, X_train_obj, img_head
        
        test_dataset = test_dataset.map(data_generator, num_threads=4, output_buffer_size=self.hps.batch_size)
        test_dataset = test_dataset.batch(1)
        
        test_iterator = Iterator.from_structure(test_dataset.output_types,
                                            test_dataset.output_shapes)
        self.test_next_element = test_iterator.get_next()
        
        self.testing_init_op = test_iterator.make_initializer(test_dataset)
        
    def _get_saver(self):
        #self.saver = tf.train.Saver(var_list=tf.trainable_variables())
        self.saver = tf.train.Saver()
        
    def save_model(self, sess, global_step):
        self.saver.save(sess, self.hps.ckpt+'cedlCNN', global_step)
        print("saved model with step %d" %(global_step))
        
    def load_model(self, sess):
        ckpt = tf.train.get_checkpoint_state(self.hps.ckpt)
        tf.logging.info('Loading model %s.', ckpt.model_checkpoint_path)
        #print('Loading model %s.', ckpt.model_checkpoint_path)
        self.saver.restore(sess, ckpt.model_checkpoint_path)
    
    def train(self):
        self.build_data()
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(self.hps.ckpt)
            if ckpt and ckpt.model_checkpoint_path:
                self.load_model(sess)
            else:
                #no checkpoint
                sess.run(tf.global_variables_initializer())
            
            best_acc = 0
            for epoch in range(self.hps.training_epochs):
                sess.run(self.training_init_op)
                loss = []
                while True:
                    try:
                        X_train, X_train_FA, X_train_ges, X_train_obj, X_train_head = sess.run(self.next_element)

                        step = sess.run(model.global_step)
                        feed_dict = {
                            self.img: X_train,
                            self.FA: X_train_FA,
                            self.ges: X_train_ges,
                            self.obj: X_train_obj,
                        }
                        _, l, step = sess.run([self.train_op, self.loss, self.global_step], feed_dict=feed_dict)
                        loss.append(l)
                    except tf.errors.OutOfRangeError:
                        print("epoch %d with total training loss %f" %(epoch, sum(loss)))
                        break
                FA_accs = []
                ges_accs = []
                obj_accs = []
                valid_losses = []
                total = 0
                FA_acc = 0
                ges_acc = 0
                obj_acc = 0
                losses = 0
                sess.run(self.validating_init_op)
                while True:
                    try:
                        X_train, X_train_FA, X_train_ges, X_train_obj, X_train_head = sess.run(self.valid_next_element)

                        feed_dict = {
                            self.img: X_train,
                            self.FA: X_train_FA,
                            self.ges: X_train_ges, 
                            self.obj:X_train_obj
                        }
                        loss, output_FA, output_ges, output_obj, step = sess.run(
                                [self.loss, self.output_FA, self.output_ges, self.output_obj, self.global_step], feed_dict=feed_dict)
                        FA_acc += np.sum(np.argmax(output_FA, axis=1)==X_train_FA.astype(int))
                        ges_acc += np.sum(np.argmax(output_ges, axis=1)==X_train_ges.astype(int))
                        obj_acc += np.sum(np.argmax(output_obj, axis=1)==X_train_obj.astype(int))
                        losses += loss*len(X_train)
                        total += len(X_train)
                    except tf.errors.OutOfRangeError:
                        losses /= self.num_valid
                        valid_losses.append(losses)
                        FA_acc = FA_acc/total
                        ges_acc = ges_acc/total
                        obj_acc = obj_acc/total
                        FA_accs.append(FA_acc)
                        ges_accs.append(ges_acc)
                        obj_accs.append(obj_acc)
                        mean_acc = np.mean([FA_acc, ges_acc, obj_acc])
                        if mean_acc > best_acc:
                            best_acc = mean_acc                
                            model.save_model(sess, step)

                        print('INFO:root:Epoch[%d] loss=%f' %(epoch, losses))
                        print('INFO:root:Epoch[%d] FA Validation-accuracy=%f' %(step, FA_acc))
                        print('INFO:root:Epoch[%d] ges Validation-accuracy=%f' %(step, ges_acc))
                        print('INFO:root:Epoch[%d] obj Validation-accuracy=%f' %(step, obj_acc))
                        print()
                        break
                FA_accs = np.array(FA_accs)
                ges_accs = np.array(ges_accs)
                obj_asss = np.array(obj_accs)
                valid_losses = np.array(valid_losses)
                np.save('FA_acc.npy', FA_accs)
                np.save('ges_acc.npy', ges_accs)
                np.save('obj_acc.npy', obj_accs)
                np.save('valid_loss.npy', valid_losses)

                
    def test(self):
        self.build_test_data()
        
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state(self.hps.ckpt)
            if ckpt and ckpt.model_checkpoint_path:
                self.load_model(sess)
            else:
                print("no chekpoint found for testing")
                return
                #sess.run(tf.global_variables_initializer())
                
            FA_accs = []
            ges_accs = []
            obj_accs = []
            #valid_losses = []
            best_acc = 0
            total = 0
            FA_acc = 0
            ges_acc = 0
            obj_acc = 0
            #losses = 0
            sess.run(self.testing_init_op)
            while True:
                try:
                    X_train, X_train_FA, X_train_ges, X_train_obj, X_train_head = sess.run(self.test_next_element)
                    feed_dict = {
                        self.img: X_train,
                    }
                    
                    
                    output_FA, output_ges, output_obj, step = sess.run(
                            [self.output_FA, self.output_ges, self.output_obj, self.global_step], feed_dict=feed_dict)
                    FA_acc += np.sum(np.argmax(output_FA, axis=1)==X_train_FA.astype(int))
                    ges_acc += np.sum(np.argmax(output_ges, axis=1)==X_train_ges.astype(int))
                    obj_acc += np.sum(np.argmax(output_obj, axis=1)==X_train_obj.astype(int))
                    #losses += loss*len(X_train)
                    total += len(X_train)
                except tf.errors.OutOfRangeError:
                    #losses /= self.num_test
                    #valid_losses.append(losses)
                    FA_acc = FA_acc/total
                    ges_acc = ges_acc/total
                    obj_acc = obj_acc/total
                    FA_accs.append(FA_acc)
                    ges_accs.append(ges_acc)
                    obj_accs.append(obj_acc)
                    mean_acc = np.mean([FA_acc, ges_acc, obj_acc])
                    if mean_acc > best_acc:
                        best_acc = mean_acc                
                        model.save_model(sess, step)

                    #print('INFO:root:Testing loss=%f' %(step, losses))
                    print('INFO:root:Testing FA-accuracy=%f' %(FA_acc))
                    print('INFO:root:Testing ges-accuracy=%f' %(ges_acc))
                    print('INFO:root:Testing obj-accuracy=%f' %(obj_acc))
                    print()
                    break


In [121]:
def get_hparams():
    hparams = tf.contrib.training.HParams(
          batch_size=64,
          lr=1e-4,
          training_epochs=10,
          w_input = 64,

          h_input = 128,
          ckpt='./model_ckpt/')
    return hparams

In [122]:
tf.reset_default_graph()
hps = get_hparams()
model = Alex_cls(hps)

In [81]:
model.train()

./frames/train
num X_train: 14992
epoch 0 with total training loss 24309492.000000
saved model with step 164
INFO:root:Epoch[0] loss=60881.778413
INFO:root:Epoch[164] FA Validation-accuracy=0.495664
INFO:root:Epoch[164] ges Validation-accuracy=0.228152
INFO:root:Epoch[164] obj Validation-accuracy=0.239715

epoch 1 with total training loss 8392971.000000
INFO:root:Epoch[1] loss=41066.290777
INFO:root:Epoch[328] FA Validation-accuracy=0.480987
INFO:root:Epoch[328] ges Validation-accuracy=0.268401
INFO:root:Epoch[328] obj Validation-accuracy=0.213920

epoch 2 with total training loss 5764279.500000
INFO:root:Epoch[2] loss=30439.293479
INFO:root:Epoch[492] FA Validation-accuracy=0.470091
INFO:root:Epoch[492] ges Validation-accuracy=0.248165
INFO:root:Epoch[492] obj Validation-accuracy=0.236602

epoch 3 with total training loss 4296810.000000
INFO:root:Epoch[3] loss=24275.771281
INFO:root:Epoch[656] FA Validation-accuracy=0.466978
INFO:root:Epoch[656] ges Validation-accuracy=0.239715
INFO:r

In [ ]:
model.test()

./frames/test
num X_test: 12776
INFO:tensorflow:Loading model ./model_ckpt/cedlCNN-1640.
INFO:tensorflow:Restoring parameters from ./model_ckpt/cedlCNN-1640
saved model with step 1640
INFO:root:Testing FA-accuracy=0.485520
INFO:root:Testing ges-accuracy=0.238181
INFO:root:Testing obj-accuracy=0.274186

